In [1]:
# Import relevant libraries from environment
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time import time
import os

# print(os.listdir("../input"))
from IPython.display import display

# Any results you write to the current directory are saved as output.
import warnings

warnings.filterwarnings("ignore")
from datetime import datetime, timedelta

In [2]:
# Load the transformed dataset
tsal_inv_df = pd.read_excel("cleaned_sal_inv_data.xlsx")
test_df = pd.read_excel("test_data.xlsx")

In [3]:
# drop the total inv column to make the dataset consistent with the test dataset
tsal_inv_df.drop(["Total inv"], inplace=True, axis=1)

In [4]:
# sorting the dataset by date
tsal_inv_df.sort_values(by="Date", inplace=True, ignore_index=True)
test_df.sort_values(by="Date", inplace=True, ignore_index=True)

In [5]:
# removing outliers from the dataset
tsal_inv_df = tsal_inv_df[tsal_inv_df["Ordered Units"] < 2500]
tsal_inv_df = tsal_inv_df[tsal_inv_df["Ordered Units"] > 0]
tsal_inv_df = tsal_inv_df[
    (tsal_inv_df["Date"] != "2022-04-13") & (tsal_inv_df["Ordered Units"] < 2000)
]
test_df = test_df[test_df["Ordered Units"] > 0]

In [6]:
# resetting the index of the dataset for later combination
test_df.reset_index(inplace=True, drop=True)
tsal_inv_df.reset_index(inplace=True, drop=True)

In [7]:
# combining the two datasets
combined_df = pd.concat([tsal_inv_df, test_df], ignore_index=True)

# First Component: obtaining 6-months average dataset and superpose it with last year same month dataset for selected features


In [8]:
# extracting dataset after 2022-05 to compute the 6 months and previous year target month average
ave_sample = combined_df[(combined_df["Date"] > "2022-05")]
nov_21_sample = combined_df[
    (combined_df["Date"] >= "2021-11-01") & (combined_df["Date"] <= "2021-11-30")
]
dec_21_sample = combined_df[
    (combined_df["Date"] >= "2021-12-01") & (combined_df["Date"] <= "2021-12-31")
]
jan_22_sample = combined_df[
    (combined_df["Date"] >= "2022-01-01") & (combined_df["Date"] <= "2022-01-31")
]

In [9]:
# resetting the index of the dataset for later combination
ave_sample.reset_index(inplace=True, drop=True)
nov_21_sample.reset_index(inplace=True, drop=True)
dec_21_sample.reset_index(inplace=True, drop=True)
jan_22_sample.reset_index(inplace=True, drop=True)

In [10]:
# combining the two datasets
nov_ave_sample = pd.concat([ave_sample, nov_21_sample], ignore_index=True)
dec_ave_sample = pd.concat([ave_sample, dec_21_sample], ignore_index=True)
jan_ave_sample = pd.concat([ave_sample, jan_22_sample], ignore_index=True)

In [11]:
# sorting the dataset by ASIN and date
nov_ave_sample.sort_values(by=["ASIN", "Date"], inplace=True, ignore_index=True)
dec_ave_sample.sort_values(by=["ASIN", "Date"], inplace=True, ignore_index=True)
jan_ave_sample.sort_values(by=["ASIN", "Date"], inplace=True, ignore_index=True)

In [12]:
# inserting the day column into the data structure
nov_ave_sample["day"] = nov_ave_sample["Date"].dt.day
dec_ave_sample["day"] = dec_ave_sample["Date"].dt.day
jan_ave_sample["day"] = jan_ave_sample["Date"].dt.day

In [13]:
def average (dataset):
    dataset = dataset.groupby(["ASIN", "day"])[
        "Ordered Units",
        "Shipped COGS",
        "Shipped COGS - % of Total",
        "Shipped Units",
        "Shipped Units - % of Total",
        "Ordered Units - % of Total",
        "Customer Returns",
        "Free Replacements",
        "Subcategory (Sales Rank)",
        "Average Sales Price",
        "Glance Views",
        "Conversion Rate",
        "Rep OOS",
        "Rep OOS - % of Total",
        "LBB (Price)",
        "AMZ Inv",
        "Furinno Inv",
    ].mean()
    return dataset

In [14]:
nov_ave_sample = average(nov_ave_sample)
dec_ave_sample = average(dec_ave_sample)
jan_ave_sample = average(jan_ave_sample)

# Second Component: obtaining prior period dataset


In [15]:
nov_pp_sample = combined_df[
    (combined_df["Date"] >= "2022-08-01") & (combined_df["Date"] <= "2022-08-31")
]
dec_pp_sample = combined_df[
    (combined_df["Date"] >= "2022-09-01") & (combined_df["Date"] <= "2022-09-30")
]
jan_pp_sample = combined_df[
    (combined_df["Date"] >= "2022-10-01") & (combined_df["Date"] <= "2022-10-31")
]

In [16]:
nov_pp_sample["day"] = nov_pp_sample["Date"].dt.day
dec_pp_sample["day"] = dec_pp_sample["Date"].dt.day
jan_pp_sample["day"] = jan_pp_sample["Date"].dt.day

In [17]:
def drop_pp (dataset):

    dataset.drop(
        [
            "Product Title",
            "Shipped COGS - % of Total",
            "Shipped COGS - Prior Period",
            "Shipped COGS - Last Year",
            "Shipped Units - % of Total",
            "Shipped Units - Prior Period",
            "Shipped Units - Last Year",
            "Ordered Units - % of Total",
            "Ordered Units - Prior Period",
            "Ordered Units - Last Year",
            "Customer Returns",
            "Free Replacements",
            "Subcategory (Sales Rank)",
            "Average Sales Price - Prior Period",
            "Change in Glance View - Prior Period",
            "Change in GV Last Year",
            "Conversion Rate",
            "Date",
            "Rep OOS - % of Total",
            "Rep OOS - Prior Period",
            "LBB (Price)",
            "UPC",
            "Model No",
            "AMZ Inv",
            "Furinno Inv",
        ],
        inplace=True,
        axis=1,
    ) 
    return dataset

In [18]:
nov_pp_sample = drop_pp(nov_pp_sample)
dec_pp_sample = drop_pp(dec_pp_sample)
jan_pp_sample = drop_pp(jan_pp_sample)

In [19]:
# renaming column name for later concatenating
def rename_pp (dataset):
    dataset.rename(
        columns={
            "Shipped COGS": "Shipped COGS - Prior Period",
            "Shipped Units": "Shipped Units - Prior Period",
            "Ordered Units": "Ordered Units - Prior Period",
            "Average Sales Price": "Average Sales Price - Prior Period",
            "Glance Views": "Change in Glance View - Prior Period",
            "Rep OOS": "Rep OOS - Prior Period",
        },
        inplace=True,
    )
    return dataset

In [20]:
nov_pp_sample = rename_pp(nov_pp_sample)
dec_pp_sample = rename_pp(dec_pp_sample)
jan_pp_sample = rename_pp(jan_pp_sample)

In [21]:
def merge (dataset_pp, dataset_ave):

    prediction_df = pd.merge(
        dataset_ave,
        dataset_pp,
        on=["ASIN", "day"],
        how="inner",
    )
    return prediction_df

In [22]:
nov_pred_df = merge(nov_pp_sample, nov_ave_sample)
dec_pred_df = merge(dec_pp_sample, dec_ave_sample)
jan_pred_df = merge(jan_pp_sample, jan_ave_sample)

In [23]:
display(nov_pred_df)
display(dec_pred_df)
display(jan_pred_df)

,ASIN,day,Ordered Units,Shipped COGS,Shipped COGS - % of Total,Shipped Units,Shipped Units - % of Total,Ordered Units - % of Total,Customer Returns,Free Replacements,...,Rep OOS - % of Total,LBB (Price),AMZ Inv,Furinno Inv,Shipped COGS - Prior Period,Shipped Units - Prior Period,Ordered Units - Prior Period,Average Sales Price - Prior Period,Change in Glance View - Prior Period,Rep OOS - Prior Period
0,B002JM8TIM,1,28.833333,636.963333,0.004333,49.833333,0.0068,0.003967,1.000000,0.333333,...,0.003750,0.083900,4086.833333,757.333333,238.02,22,23,28.70,637,0.0062
1,B002JM8TIM,2,29.142857,280.997143,0.002243,22.857143,0.0034,0.003771,0.857143,0.714286,...,0.003829,0.143686,4257.428571,152.714286,216.72,20,19,28.70,610,0.0048
2,B002JM8TIM,3,31.142857,363.982857,0.002514,28.857143,0.0039,0.004143,0.714286,0.142857,...,0.003971,0.133286,5658.142857,93.142857,216.72,21,28,28.70,684,0.0029
3,B002JM8TIM,4,33.285714,332.151429,0.002657,27.857143,0.0041,0.004400,1.000000,0.000000,...,0.003443,0.071214,5799.000000,84.857143,268.32,26,24,28.70,565,0.0035
4,B002JM8TIM,5,25.571429,355.888571,0.002671,30.571429,0.0043,0.003043,2.285714,0.285714,...,0.007371,0.071143,5912.142857,77.714286,247.68,24,26,28.70,569,0.0035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17018,B0B3SCWY7D,20,2.500000,124.530000,0.000850,3.000000,0.0004,0.000300,0.000000,0.000000,...,0.000000,0.005600,25.500000,406.000000,84.60,2,1,57.53,27,0.0000
17019,B0B5Y1R8R5,3,1.000000,54.300000,0.000300,1.000000,0.0001,0.000100,0.000000,0.000000,...,0.000000,0.000000,0.000000,346.000000,54.30,1,1,87.65,1,0.0000
17020,B0B5Y1R8R5,19,1.500000,81.450000,0.000500,1.000000,0.0001,0.000200,0.000000,0.500000,...,0.000000,0.000000,2.500000,400.500000,54.30,1,1,74.90,16,0.0000
17021,B0B5Y21FGQ,11,1.000000,22.200000,0.000100,1.000000,0.0001,0.000100,0.000000,0.000000,...,0.000000,0.000000,0.000000,399.000000,22.20,1,1,35.59,1,0.0000


,ASIN,day,Ordered Units,Shipped COGS,Shipped COGS - % of Total,Shipped Units,Shipped Units - % of Total,Ordered Units - % of Total,Customer Returns,Free Replacements,...,Rep OOS - % of Total,LBB (Price),AMZ Inv,Furinno Inv,Shipped COGS - Prior Period,Shipped Units - Prior Period,Ordered Units - Prior Period,Average Sales Price - Prior Period,Change in Glance View - Prior Period,Rep OOS - Prior Period
0,B002JM8TIM,1,24.833333,326.963333,0.002317,30.000000,0.004033,0.003167,1.000000,0.166667,...,0.002900,0.083900,4086.833333,1277.666667,753.36,72,46,24.30,977,0.0279
1,B002JM8TIM,2,23.000000,216.454286,0.001586,19.000000,0.002586,0.002714,0.428571,0.428571,...,0.002700,0.143686,4257.285714,604.285714,319.92,29,51,24.30,890,0.0294
2,B002JM8TIM,3,27.142857,228.747143,0.001386,20.142857,0.002443,0.003329,0.571429,0.142857,...,0.003414,0.133286,5658.000000,552.142857,371.52,36,64,24.30,1090,0.0294
3,B002JM8TIM,4,30.000000,248.010000,0.001850,23.500000,0.003217,0.003600,1.000000,0.000000,...,0.001250,0.083083,6765.000000,81.333333,712.08,69,85,24.30,1336,0.0111
4,B002JM8TIM,5,27.000000,267.892857,0.001814,24.857143,0.003200,0.003143,2.285714,0.285714,...,0.001700,0.071143,5912.285714,551.428571,712.08,68,68,24.30,1332,0.0052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13356,B0BB7NRV9D,23,11.000000,269.500000,0.002000,7.000000,0.001000,0.001500,0.000000,0.000000,...,0.000000,0.000000,0.000000,33.000000,269.50,7,11,57.38,279,0.0000
13357,B0BB7NRV9D,26,6.000000,693.000000,0.003500,18.000000,0.001900,0.000800,0.000000,0.000000,...,0.000000,0.000000,1.000000,13.000000,693.00,18,6,57.38,272,0.0000
13358,B0BB7NRV9D,27,4.000000,173.250000,0.001200,4.500000,0.000600,0.000550,0.500000,0.000000,...,0.000000,0.000000,7.000000,118.500000,308.00,8,7,57.38,250,0.0000
13359,B0BB7NRV9D,28,3.000000,115.500000,0.000900,3.000000,0.000500,0.000400,1.000000,0.000000,...,0.000000,0.000000,2.000000,2.000000,115.50,3,3,57.38,180,0.0000


,ASIN,day,Ordered Units,Shipped COGS,Shipped COGS - % of Total,Shipped Units,Shipped Units - % of Total,Ordered Units - % of Total,Customer Returns,Free Replacements,...,Rep OOS - % of Total,LBB (Price),AMZ Inv,Furinno Inv,Shipped COGS - Prior Period,Shipped Units - Prior Period,Ordered Units - Prior Period,Average Sales Price - Prior Period,Change in Glance View - Prior Period,Rep OOS - Prior Period
0,B002JM8TIM,1,26.000000,342.756000,0.002480,32.800000,0.004440,0.003200,1.000000,0.200000,...,0.003080,0.100680,4903.200000,848.600000,567.60,55,46,24.3,1002,0.0243
1,B002JM8TIM,2,34.000000,170.118571,0.001257,16.000000,0.002171,0.003643,0.285714,0.428571,...,0.003071,0.143686,4257.142857,478.857143,485.04,47,64,24.3,1178,0.0273
2,B002JM8TIM,3,32.000000,707.197143,0.002986,51.000000,0.004443,0.003743,0.428571,0.142857,...,0.003771,0.133286,5657.714286,395.285714,843.84,68,53,24.3,1024,0.0201
3,B002JM8TIM,4,33.428571,352.080000,0.002229,29.142857,0.003514,0.004000,1.000000,0.000000,...,0.002771,0.071214,5798.857143,384.857143,435.42,39,52,24.3,940,0.0032
4,B002JM8TIM,5,31.285714,409.771429,0.002543,34.000000,0.004086,0.003657,2.571429,0.285714,...,0.002129,0.071143,5912.285714,375.285714,612.18,53,42,24.3,951,0.0135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149,B0BF1FC4HQ,22,1.000000,196.000000,0.001800,1.000000,0.000200,0.000100,0.000000,0.000000,...,0.000000,0.000000,21.000000,143.000000,196.00,1,1,237.1,196,0.0000
15150,B0BF1FC4HQ,27,2.000000,588.000000,0.003900,3.000000,0.000400,0.000300,0.000000,0.000000,...,0.000000,0.000000,18.000000,382.000000,588.00,3,2,237.1,177,0.0000
15151,B0BF1FC4HQ,28,1.000000,392.000000,0.002600,2.000000,0.000300,0.000100,0.000000,0.000000,...,0.000000,0.000000,17.000000,382.000000,392.00,2,1,237.1,147,0.0000
15152,B0BF1FC4HQ,30,2.000000,196.000000,0.001200,1.000000,0.000100,0.000200,0.000000,0.000000,...,0.000000,0.000000,16.000000,382.000000,196.00,1,2,237.1,268,0.0000


# Third Component: obtaining last year dataset


In [24]:
# obtain last year dataset
nov_ly_sample = combined_df[
    (combined_df["Date"] >= "2021-11-01") & (combined_df["Date"] <= "2021-11-30")
]
dec_ly_sample = combined_df[
    (combined_df["Date"] >= "2021-12-01") & (combined_df["Date"] <= "2021-12-31")
]
jan_ly_sample = combined_df[
    (combined_df["Date"] >= "2022-01-01") & (combined_df["Date"] <= "2022-01-31")
]

In [25]:
nov_ly_sample["day"] = nov_ly_sample["Date"].dt.day
dec_ly_sample["day"] = dec_ly_sample["Date"].dt.day
jan_ly_sample["day"] = jan_ly_sample["Date"].dt.day


In [26]:
def drop_ly (dataset):
    dataset.drop(
        [
            "Product Title",
            "Shipped COGS - % of Total",
            "Shipped COGS - Prior Period",
            "Shipped COGS - Last Year",
            "Shipped Units - % of Total",
            "Shipped Units - Prior Period",
            "Shipped Units - Last Year",
            "Ordered Units - % of Total",
            "Ordered Units - Prior Period",
            "Ordered Units - Last Year",
            "Customer Returns",
            "Free Replacements",
            "Subcategory (Sales Rank)",
            "Average Sales Price - Prior Period",
            "Average Sales Price",
            "Change in Glance View - Prior Period",
            "Change in GV Last Year",
            "Conversion Rate",
            "Rep OOS",
            "Rep OOS - % of Total",
            "Rep OOS - Prior Period",
            "LBB (Price)",
            "UPC",
            "Model No",
            "AMZ Inv",
            "Furinno Inv",
        ],
        inplace=True,
        axis=1,
    )
    return dataset

In [27]:
nov_ly_sample = drop_ly(nov_ly_sample)
dec_ly_sample = drop_ly(dec_ly_sample)
jan_ly_sample = drop_ly(jan_ly_sample)

In [28]:
# renaming column name for later concatenating
def rename_ly (dataset):

    dataset.rename(
        columns={
            "Shipped COGS": "Shipped COGS - Last Year",
            "Shipped Units": "Shipped Units - Last Year",
            "Ordered Units": "Ordered Units - Last Year",
            "Average Sales Price": "Average Sales Price - Last Year",
            "Glance Views": "Change in GV Last Year",
        },
        inplace=True,
    )
    return dataset

In [29]:
# renaming all the columns
nov_ly_sample = rename_ly(nov_ly_sample)
dec_ly_sample = rename_ly(dec_ly_sample)
jan_ly_sample = rename_ly(jan_ly_sample)


In [30]:
# merging the dataset with the last year dataset to form the final prediction data
def merge_ly (dataset, dataset_ly):
    
    prediction = pd.merge(
        dataset, dataset_ly, on=["ASIN", "day"], how="inner"
    )
    return prediction

In [31]:
nov_pred_df = merge_ly(nov_pred_df, nov_ly_sample)
dec_pred_df = merge_ly(dec_pred_df, dec_ly_sample)
jan_pred_df = merge_ly(jan_pred_df, jan_ly_sample)

# Final transformation 

In [32]:
# Convert datetime object to string in format YYYY-MM-DD
nov_pred_df["Date"] = nov_pred_df["Date"].dt.strftime(
    "%Y-%m-%d"
)
dec_pred_df["Date"] = dec_pred_df["Date"].dt.strftime(
    "%Y-%m-%d"
)  
jan_pred_df["Date"] = jan_pred_df["Date"].dt.strftime(
    "%Y-%m-%d"
)

# Replace all occurrences of "2021" with "2022"
nov_pred_df["Date"] = nov_pred_df["Date"].str.replace(
    "2021", "2022"
)
dec_pred_df["Date"] = dec_pred_df["Date"].str.replace(
    "2021", "2022"
)
jan_pred_df["Date"] = jan_pred_df["Date"].str.replace(
    "2021", "2022"
)

# Convert back to pandas datetime object
nov_pred_df["Date"] = pd.to_datetime(
    nov_pred_df["Date"]
)
dec_pred_df["Date"] = pd.to_datetime(
    dec_pred_df["Date"]
)
jan_pred_df["Date"] = pd.to_datetime(
    jan_pred_df["Date"]
)

In [33]:
nov_pred_df.drop(["day"], inplace=True, axis=1)
dec_pred_df.drop(["day"], inplace=True, axis=1)
jan_pred_df.drop(["day"], inplace=True, axis=1)


In [34]:
nov_pred_df.to_excel("nov_pred_df.xlsx")
dec_pred_df.to_excel("dec_pred_df.xlsx")
jan_pred_df.to_excel("jan_pred_df.xlsx")


In [41]:
jan_pred_df

,ASIN,Ordered Units,Shipped COGS,Shipped COGS - % of Total,Shipped Units,Shipped Units - % of Total,Ordered Units - % of Total,Customer Returns,Free Replacements,Subcategory (Sales Rank),...,Shipped Units - Prior Period,Ordered Units - Prior Period,Average Sales Price - Prior Period,Change in Glance View - Prior Period,Rep OOS - Prior Period,Shipped COGS - Last Year,Shipped Units - Last Year,Ordered Units - Last Year,Change in GV Last Year,Date
0,B002JM8TIM,34.000000,170.118571,0.001257,16.000000,0.002171,0.003643,0.285714,0.428571,167.428571,...,47,64,24.30,1178,0.0273,14.35,1,91,2426,2022-01-02
1,B002JM8TIM,32.000000,707.197143,0.002986,51.000000,0.004443,0.003743,0.428571,0.142857,122.285714,...,68,53,24.30,1024,0.0201,3363.50,217,64,1637,2022-01-03
2,B002JM8TIM,33.428571,352.080000,0.002229,29.142857,0.003514,0.004000,1.000000,0.000000,130.571429,...,39,52,24.30,940,0.0032,976.50,63,54,1561,2022-01-04
3,B002JM8TIM,31.285714,409.771429,0.002543,34.000000,0.004086,0.003657,2.571429,0.285714,85.571429,...,53,42,24.30,951,0.0135,1007.50,65,60,1681,2022-01-05
4,B002JM8TIM,34.000000,420.271429,0.002286,34.000000,0.003657,0.004029,1.714286,0.428571,143.428571,...,58,65,24.30,815,0.0049,862.82,56,54,1704,2022-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9173,B09PG3CN5W,11.833333,149.626667,0.001217,5.333333,0.000767,0.001517,0.333333,0.000000,151.333333,...,5,9,52.80,985,0.0000,39.73,1,26,992,2022-01-29
9174,B09PG3CN5W,9.166667,157.828333,0.001200,6.166667,0.000817,0.001083,0.666667,0.000000,150.333333,...,11,16,50.36,1156,0.0000,39.73,1,20,1090,2022-01-30
9175,B09PG3CN5W,13.750000,784.737500,0.003950,21.750000,0.002200,0.001775,0.500000,0.000000,72.000000,...,14,17,49.82,862,0.0000,2105.69,53,22,871,2022-01-31
9176,B09PG3QTBC,3.500000,214.316667,0.001717,3.166667,0.000450,0.000433,0.500000,0.166667,262.000000,...,9,12,59.94,1109,0.0000,72.73,1,1,26,2022-01-25


In [35]:
jan_pred_df[jan_pred_df["Ordered Units"] > 2500]

,ASIN,Ordered Units,Shipped COGS,Shipped COGS - % of Total,Shipped Units,Shipped Units - % of Total,Ordered Units - % of Total,Customer Returns,Free Replacements,Subcategory (Sales Rank),...,Shipped Units - Prior Period,Ordered Units - Prior Period,Average Sales Price - Prior Period,Change in Glance View - Prior Period,Rep OOS - Prior Period,Shipped COGS - Last Year,Shipped Units - Last Year,Ordered Units - Last Year,Change in GV Last Year,Date


In [36]:
# get particular index information
def get_index (dataset, index):
    dataset = dataset.loc[index]
    return dataset

In [37]:
get_index(nov_pred_df, 3)

ASIN                                             B002JM8TIM
Ordered Units                                     33.285714
Shipped COGS                                     332.151429
Shipped COGS - % of Total                          0.002657
Shipped Units                                     27.857143
Shipped Units - % of Total                           0.0041
Ordered Units - % of Total                           0.0044
Customer Returns                                        1.0
Free Replacements                                       0.0
Subcategory (Sales Rank)                         129.857143
Average Sales Price                               29.758571
Glance Views                                     701.428571
Conversion Rate                                    0.038629
Rep OOS                                            0.021386
Rep OOS - % of Total                               0.003443
LBB (Price)                                        0.071214
AMZ Inv                                 

In [38]:
# get all the row that have NaN ordered units value
def get_nan (dataset):
    dataset = dataset[dataset["Ordered Units"].isna()]
    return dataset